# infos

In [ ]:
# Version 1.1

# Notebook test d'une optimisation optuna

### Changelogs :

v1.2 :
 - Mise en jour du code en adéquation avec les fonctions mises à jour dans la v1.2 de 3_CNN


v1.1 :
- Nettoyage du code pour livraison projet
- Itération d'un modèle


v1.0 :
- Construction du notebook
- Definition des cellules Optuna

# Definition des URL

In [13]:
# Vérifier que les chemins soient correct avant toutes opérations

# Définition du dossiers contenant les images
chemin_images = '../../images/'

# Définition du fichier .csv utilisé (décommenter la ligne souhaitée)
chemin_csv = '../data/top10.csv'              # Les images sont triées
#chemin_csv = '../data/top10_no_tri.csv'      # Les images ne sont pas triées

# Dimensions retenues des images en entrée du modèle
img_dim = (200,200)
img_shape = (200,200,3)


# Choix des données d'entrainement :
pourcentage_echantillon = 0.1 # 0.1 représente 10% de la masse de données

# (Decommenter la ligne souhaitée)

# si 'oui', les données d'entrainement ne seront basées que sur un echantillon :
set_echantillon = 'oui'
#set_echantillon = 'non'

undersampling = 'oui'
#undersampling = 'non'

# Google Colab

Si le notebook tourne sur colab, charger les fichiers images (format zip) et dezipper en suivant les cellules qui suivent :

In [ ]:
# Importer les images en format .zip
from google.colab import files
files.upload()

In [2]:
# Monter le Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Dezipper le fichier
!unzip '/content/drive/MyDrive/SAS/images.zip' -d '/images'

Streaming output truncated to the last 5000 lines.
  inflating: /images/images/672562.jpg  
  inflating: /images/images/672563.jpg  
  inflating: /images/images/672564.jpg  
  inflating: /images/images/67257.jpg  
  inflating: /images/images/672570.jpg  
  inflating: /images/images/67258.jpg  
  inflating: /images/images/672580.jpg  
  inflating: /images/images/672584.jpg  
  inflating: /images/images/672585.jpg  
  inflating: /images/images/672586.jpg  
  inflating: /images/images/672587.jpg  
  inflating: /images/images/67259.jpg  
  inflating: /images/images/672602.jpg  
  inflating: /images/images/67261.jpg  
  inflating: /images/images/67264.jpg  
  inflating: /images/images/672656.jpg  
  inflating: /images/images/672657.jpg  
  inflating: /images/images/672664.jpg  
  inflating: /images/images/672665.jpg  
  inflating: /images/images/672666.jpg  
  inflating: /images/images/672667.jpg  
  inflating: /images/images/672668.jpg  
  inflating: /images/images/672703.jpg  
  inflating

In [31]:
# Remplacer les chemins en corrélation avec les dossiers colab
chemin_images = '/images/images/'
chemin_csv = '/content/drive/MyDrive/SAS/Jul23_bds_champignons/data/top10.csv'

In [5]:
# Pour installation sur Colab
!pip install --quiet optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 10.9 MB/s eta 0:00:00


# Librairies à charger

In [14]:
# Librairies générales servant dans le notebook
import pandas as pd
import os


# Librairies appelées pour l'utilisatio ndes fonctions définies dans la partie 'fonctions
from tensorflow.keras.applications.efficientnet import preprocess_input


# Librairies utilisées pour les callbacks
from tensorflow.keras import callbacks
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import ModelCheckpoint
from timeit import default_timer as timer
from tensorflow.keras.callbacks import TerminateOnNaN


# Librairies utilisées pour créer les pipelines et le mµodèle
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense


# Librairies utilisées pour la création des jeux d'entrainement, de test et de validation
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from collections import Counter


# Librairies utilisées pour l'évaluation du modèle une fois entrainé
import matplotlib.pyplot as plt
%matplotlib inline


# Librairies utilisées pour des sauvegardes et chargements de variables
import pickle

# Librairie OPTUNA
import optuna


# Fonctions

In [15]:
def import_df(chemin_images, chemin_csv, pourcentage_echantillon = 0.1):
    '''
    Importe le fichier csv et construit 2 df :
        - Le DF basé sur le CSV original
        - Un DF echantillon comportant 10% de données aléatoires du DF original

    Arguments :
        - chemin_images : Chemin vers le dossier images
        - chemin_csv : Chemin vers le fichier .csv contenant les données utilisées
        pourcentage_echantillon : Taille du DF echantillon tiré du DF original
    '''


    # import du df
    df = pd.read_csv(chemin_csv, low_memory=False)
    df['image_url'] = df['image_url'].str.replace('.../images/', chemin_images)
    print(f"Nombre d'images chargées pour df: {df.shape[0]}")
    print(f"Nb especes dans df: {df['label'].nunique()}")


    # Contruction de l'echantillon
    L = len(df)
    L_ech = int(pourcentage_echantillon * L)
    df_ech = df.sample(n=L_ech, random_state=10)
    df_ech.reset_index(inplace=True, drop=True)
    print(f"Nombre d'images chargées pour df_ech: {df_ech.shape[0]}")
    print(f"Nb especes dans df_ech: {df_ech['label'].nunique()}")


    return df, df_ech

In [16]:
def undersampling_df(df, col):

    '''
    Undersample le df donné pour équilibrer le nombre d'pobservations par classe.
        - df : df à undersampler
        - col : colonne concernée par le GroupBy pour générer l'undersampling
    '''

    compte = df.groupby(col).count()
    min_samples = compte['image_url'].min()
    min_samples = int(min_samples)

    df_undersample = pd.DataFrame()

    for label, group in df.groupby('label'):
        df_undersample = pd.concat([df_undersample, group.sample(min_samples, replace=True)])
        df_undersample = df_undersample.reset_index(drop=True)

    return df_undersample

In [17]:
def augment_img(image_path, label):

   '''
    Modifie les images aléatoirement dans le dataset qui sera soumis au modèle.
      - image_path : URL des images (contenue dans la variable 'image_url' dans le DF chargé),
      - label : Variable contenant les classes
   '''

   # Lecture image, decodage
   img = tf.io.read_file(image_path)
   img = tf.image.decode_png(img, channels=3)

   # Redimensionnement selon dimensions définies en début de notebook (img_dim)
   img = tf.image.resize(img, img_dim)

   # Pre-processing pour transfert learning, modèle efficienNet
   img = preprocess_input(img)

   # Augmentations aléatoires des images :

   # Inversion Gauche/Droite
   img = tf.image.random_flip_left_right(img)

   # Inversion Haut/Bas
   img = tf.image.random_flip_up_down(img)

   # Modification luminosité
   img = tf.image.random_brightness(img, max_delta=0.2)

   # Modification contraste
   img = tf.image.random_contrast(img, lower=0.8, upper=1.2)

   # Conversion du type en float32
   img = tf.image.convert_image_dtype(img, tf.float32)

   # Normalisation
   img = (img - tf.math.reduce_min(img)) / (tf.math.reduce_max(img) - tf.math.reduce_min(img))

   return img, label

In [18]:
def create_tf_dataset(image_path, labels, batch_size):
    '''
    Créé un dataset Tensorflow selon les paramètres précisés.
        - image_path : chemin relatif de la variable contenant les images
        - labels : variable contenant les labels
        - batch_size : taille des batchs
    '''

    image_path = image_path.tolist()  # Convertir les chemins d'images en liste
    labels = labels.tolist()          # Convertir les labels en liste



    # Construction du Dataset
    dataset = tf.data.Dataset.from_tensor_slices((image_path, labels))

    # .map appelle la fonction d'augmentation d'image définie
    dataset = dataset.map(augment_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)

    # Mélange aléatoire du dataset
    dataset = dataset.shuffle(buffer_size=len(image_path))

    # Découpage en batch
    dataset = dataset.batch(batch_size)

    # Optimisation : Charge les données en arrière-plan et maintien la charge CPU/GPU
    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    return dataset

In [19]:
def controle_presence_fichiers(df, chemin_images):

    '''
    Controle que les fichiers images soient bien présents sur le disque.
        - df : DataFrame contenant les url des fichiers images
        - chemin_images : Variable du DF contenant les url
    '''

    image_directory = chemin_images
    missing_files = []

    # Parcourir chaque ligne du DataFrame
    for index, row in df.iterrows():
        image_path = os.path.join(image_directory, row['image_lien'])

        if not os.path.exists(image_path):
            missing_files.append(image_path)

    # Afficher les fichiers non trouvés
    if missing_files:
        print("\nFichiers non trouvés :")
        for file_path in missing_files:
            print(file_path)

    # Ou préciser que tous les fichiers sont présents
    else:
        print("\nTous les fichiers sont présents.")

# Optuna - Fonctions

### Fonction creation modele

In [46]:
def create_model(trial):
    # Paramètres suggérés pour l'optimisation
    num_hidden_layers = trial.suggest_int('num_hidden_layers', 1, 5)
    num_hidden_units = trial.suggest_int('num_hidden_units', 64, 256)
    dropout_rate = trial.suggest_uniform('dropout_rate', 0.0, 0.5)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)



    # Modèle pré-entraîné
    efficientNetv2 = "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_b0/classification/2"
    pre_trained_model = hub.KerasLayer(efficientNetv2, input_shape=img_shape, trainable=False)


    model = keras.Sequential([pre_trained_model,
                              layers.Reshape((1,1,1000)),
                              GlobalAveragePooling2D()])

    for _ in range(num_hidden_layers):
        model.add(Dropout(dropout_rate))
        model.add(Dense(num_hidden_units, activation='relu'))

    # Couche de sortie avec 10 classes
    model.add(Dense(10, activation='softmax'))

    # Compilation du modèle
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

### Fonction objectif

In [22]:
def objective(trial):
    # Créer le modèle
    model = create_model(trial)

    # Entraîner le modèle
    history = model.fit(ds_train,
                        validation_data = ds_val,
                        epochs=15,
                        callbacks = [early_stopping, reduceLR, checkpoint, time_callback, TON],
                        batch_size=batch_size)

    # Récupérer la précision sur la validation finale
    final_val_accuracy = history.history['val_accuracy'][-1]

    # Mettre à jour l'étude Optuna avec la métrique de précision
    trial.report(final_val_accuracy, step=0)

    # Indiquer à Optuna si l'objectif a été atteint (maximisation de la précision)
    if trial.should_prune():
        raise optuna.TrialPruned()

    return final_val_accuracy

### Etude

In [44]:
study = optuna.create_study(direction='maximize')

[I 2023-09-20 17:33:35,353] A new study created in memory with name: no-name-a8d2b582-4dea-4e1f-9ac6-bcf6d2ecf205


# Callbacks

### EarlyStopping

In [24]:
early_stopping = EarlyStopping(monitor = 'val_accuracy',
                               min_delta = 0.03,
                               patience = 8,
                               verbose = 1,
                               mode = 'auto',
                               restore_best_weights = True)

### Reduce LearningRate

In [25]:
reduceLR = ReduceLROnPlateau(monitor = 'val_loss',
                             min_delta = 0.01,
                             patience = 5,
                             factor = 0.15,
                             cooldown = 3,
                             verbose = 1)

### Checkpoint

In [26]:
checkpoint = ModelCheckpoint(filepath='../model/checkpoint_model', monitor='val_accuracy', save_best_only=True, verbose=1)

### Timer

In [27]:
class TimingCallback(Callback):
    def __init__(self):
        super().__init__()
        self.logs = []

    def on_epoch_begin(self, epoch, logs=None):
        self.starttime = timer()

    def on_epoch_end(self, epoch, logs=None):
        endtime = timer()
        elapsed_time = endtime - self.starttime
        self.logs.append(elapsed_time)
        print(f"Epoch {epoch + 1} took {elapsed_time:.2f} seconds")

time_callback = TimingCallback()

### Terminate on NaN

In [28]:
TON = TerminateOnNaN()

# Pipeline Dataset

### Import des DataFrames

In [32]:
df, df_ech = import_df(chemin_images, chemin_csv, pourcentage_echantillon)

Nombre d'images chargées pour df: 60481
Nb especes dans df: 10
Nombre d'images chargées pour df_ech: 6048
Nb especes dans df_ech: 10


<ipython-input-15-b02316a4252b>:16: FutureWarning: The default value of regex will change from True to False in a future version.
  df['image_url'] = df['image_url'].str.replace('.../images/', chemin_images)


In [33]:
# Préciser en début de notebook sur quelles données travaillées

if set_echantillon == 'oui':
  donnees_training = df_ech
else:
  donnees_training = df

In [34]:
# Controle de la présence des fichiers images
controle_presence_fichiers(donnees_training, chemin_images)

# On supprime ensuite la colonne image_lien qui ne sert qu'à controler la présence des fichiers.
donnees_training.drop('image_lien', axis=1, inplace=True)


Tous les fichiers sont présents.


In [35]:
donnees_training.head()

,label,image_url
0,Agaricales,../images/images/408705.jpg
1,Cortinarius,../images/images/303020.jpg
2,Russula,../images/images/462695.jpg
3,Amanita,../images/images/54266.jpg
4,Cortinarius,../images/images/356657.jpg


In [37]:
# undersampling des classes : (si défini)
if undersampling == 'oui':
  donnees_training = undersampling_df(donnees_training, col = 'label')
  print(donnees_training.groupby('label').count())

else:
  print("Pas d'undersampling programmé, les données sont déséquilibrées.")

             image_url
label                 
Agaricales         387
Agaricus           387
Amanita            387
Cortinarius        387
Entoloma           387
Inocybe            387
Mycena             387
Polyporales        387
Psathyrella        387
Russula            387


### Construction des jeux de données (train, test et validation)

In [38]:
data = donnees_training.drop('label', axis=1)
target = donnees_training['label']

s = LabelEncoder()
target = s.fit_transform(target) # Encodage de la variable 'label'

# On construit le jeu d'entrainnement. X_temp et y_temps servent pour la construction des jeux de test et validation
X_train, X_temp, y_train, y_temp = train_test_split(data, target, test_size=0.25, random_state=10)

# On split les temp en 50% pour test, 50% pour validation
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=10)

### Construction des dataset Tensorflow

In [39]:
batch_size = 32
# Les datasets sont créés à partir de la fonction create_tf_dataset définie dans la partie 'Fonctions'
ds_train= create_tf_dataset(X_train.image_url, y_train, batch_size)
ds_val = create_tf_dataset(X_val.image_url, y_val, batch_size)

# Optuna - Otimisation

Utilisez la méthode optimize de l'objet d'étude en spécifiant la fonction objectif et le nombre d'essais que vous souhaitez effectuer

In [47]:
# /!\ Attention l'entrainement peut être très très long
from tensorflow import keras
from tensorflow.keras.optimizers import Adam


study.optimize(objective, n_trials=10, n_jobs=-1)

<ipython-input-46-6ef3f0541bf3>:5: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_rate = trial.suggest_uniform('dropout_rate', 0.0, 0.5)
<ipython-input-46-6ef3f0541bf3>:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)


Epoch 1/15
Epoch 1/15
Epoch 1/15
Epoch 1/15
Epoch 1/15
Epoch 1/15
Epoch 1/15
Epoch 1/15
 1/91 [..............................] - ETA: 1:50:30 - loss: 2.6697 - accuracy: 0.0938

91/91 [==============================] - ETA: 0s - loss: 2.0438 - accuracy: 0.2557
Epoch 1: val_accuracy improved from -inf to 0.10537, saving model to ../model/checkpoint_model

Epoch 1: val_accuracy improved from 0.10537 to 0.20041, saving model to ../model/checkpoint_model

Epoch 1: val_accuracy improved from 0.20041 to 0.36570, saving model to ../model/checkpoint_model

Epoch 1: val_accuracy did not improve from 0.20041
Epoch 1 took 80.79 seconds
91/91 [==============================] - 90s 178ms/step - loss: 2.4072 - accuracy: 0.1402 - val_loss: 2.2399 - val_accuracy: 0.1818 - lr: 5.1058e-05
Epoch 2/15

Epoch 1: val_accuracy did not improve from 0.36570
Epoch 1 took 0.01 seconds
91/91 [==============================] - 90s 179ms/step - loss: 2.5364 - accuracy: 0.1034 - val_loss: 2.2575 - val_accuracy: 0.1632 - lr: 5.1058e-05
Epoch 2/15

Epoch 1: val_accuracy improved from 0.36570 to 0.41116, saving model to ../model/checkpoint_model

Epoch 1: val_accuracy did not improve from 0.41

[W 2023-09-20 17:37:24,734] Trial 10 failed with parameters: {'num_hidden_layers': 3, 'num_hidden_units': 214, 'dropout_rate': 0.23270736728190328, 'learning_rate': 0.0017904760772844135} because of the following error: TypeError("Binding inputs to tf.function `keras_layer_layer_call_and_return_conditional_losses` failed due to `missing a required argument: 'inputs'`. Received args: (<tf.Tensor 'keras_layer_input:0' shape=(None, 200, 200, 3) dtype=float32>,) and kwargs: {'training': True} for signature: (self, inputs, training=<object object at 0x79411be33480>).").
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-22-2f32fad3a451>", line 6, in objective
    history = model.fit(ds_train,
  File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
    raise e.with_traceback(filtered_tb) from None
 

71/91 [======================>.......] - ETA: 1s - loss: 2.3505 - accuracy: 0.1338
Epoch 2: ReduceLROnPlateau reducing learning rate to 7.658714639546814e-06.
44/91 [=============>................] - ETA: 3s - loss: 2.0403 - accuracy: 0.2443
Epoch 2: val_accuracy did not improve from 0.41116
13/91 [===>..........................] - ETA: 5s - loss: 2.1600 - accuracy: 0.2139
Epoch 2: val_accuracy did not improve from 0.41116
Epoch 2 took 2.10 seconds
49/91 [===============>..............] - ETA: 2s - loss: 2.1448 - accuracy: 0.2130
Epoch 2: val_accuracy did not improve from 0.41116
Epoch 2 took 1.44 seconds
65/91 [====================>.........] - ETA: 1s - loss: 2.1409 - accuracy: 0.2091
Epoch 2: val_accuracy improved from 0.41116 to 0.45661, saving model to ../model/checkpoint_model


...

11/91 [==>...........................] - ETA: 2s - loss: 2.1515 - accuracy: 0.2074
Epoch 7: val_accuracy did not improve from 0.49587
Epoch 7 took 1.60 seconds
62/91 [===================>..........] - ETA: 0s - loss: 1.9524 - accuracy: 0.2802
Epoch 7: val_accuracy did not improve from 0.49587
Epoch 7 took 3.02 seconds
91/91 [==============================] - 5s 39ms/step - loss: 2.0917 - accuracy: 0.2336 - val_loss: 1.9876 - val_accuracy: 0.3140 - lr: 1.7232e-07
Epoch 8/15
 3/91 [..............................] - ETA: 2s - loss: 2.1210 - accuracy: 0.2812  
Epoch 8: val_accuracy did not improve from 0.49587
Epoch 8 took 1.55 seconds
91/91 [==============================] - 5s 40ms/step - loss: 1.9723 - accuracy: 0.2750 - val_loss: 1.9095 - val_accuracy: 0.3182 - lr: 1.7232e-07
Epoch 9/15
56/91 [=================>............] - ETA: 1s - loss: 2.0120 - accuracy: 0.2561
Epoch 8: val_accuracy did not improve from 0.49587
Epoch 8 took 3.31 seconds
11/91 [==>...........................] - E

[I 2023-09-20 17:38:42,213] Trial 8 finished with value: 0.3099173605442047 and parameters: {'num_hidden_layers': 3, 'num_hidden_units': 104, 'dropout_rate': 0.49925792036544847, 'learning_rate': 0.0063172492999284515}. Best is trial 8 with value: 0.3099173605442047.


91/91 [==============================] - ETA: 0s - loss: 1.7958 - accuracy: 0.3484Restoring model weights from the end of the best epoch: 6.

Epoch 15: val_accuracy did not improve from 0.49587
Epoch 15 took 3.44 seconds
91/91 [==============================] - 3s 29ms/step - loss: 1.7958 - accuracy: 0.3484 - val_loss: 1.6494 - val_accuracy: 0.4380 - lr: 3.8772e-09
Epoch 15: early stopping


[I 2023-09-20 17:38:45,485] Trial 14 finished with value: 0.43801653385162354 and parameters: {'num_hidden_layers': 3, 'num_hidden_units': 70, 'dropout_rate': 0.3695013434616025, 'learning_rate': 0.00023083081686847787}. Best is trial 14 with value: 0.43801653385162354.


TypeError: ignored

# Optuna - Resultats

Une fois l'optimisation terminée, vous pouvez accéder aux meilleurs hyperparamètres et à la meilleure valeur obtenue

In [ ]:
best_params = study.best_params
best_value = study.best_value
print(best_params)

In [ ]:
optuna_viz.plot_param_importances(study)

# Optuna - Construction best_model

In [ ]:
# Compiler le modèle
best_model = create_model(study.best_trial)

In [ ]:
best_model.summary()

# Sauvegarde du modèle

In [ ]:
# Changer le nom du modèle si il s'agit d'un nouvel entrainement

# Save en dur ou sur Gdrive
#nom_modele = '../model/gpot_v03_optuna_full_tri_undersampled'                          # Save en dur
nom_modele =  '/content/drive/MyDrive/SAS/model/gpot_v03_optuna_full_tri_undersampled'     # Save Gdrive


# historique d'entrainement
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
history = {'val_accuracy': val_accuracy, 'accuracy': accuracy}


gpot_v03_optuna_full_tri_undersampled = history

# Save en dur
# chemin_acces = '../history/gpot_v03_optuna_full_tri_undersampled.pkl'

# Save sur GDrive
chemin_acces ='/content/drive/MyDrive/SAS/history/gpot_v03_optuna_full_tri_undersampled.pkl'

In [ ]:
model.save(nom_modele)